In [ ]:
import scraper
import pandas as pd
import numpy as np
from datetime import datetime
import json

In [ ]:
df = pd.read_csv('data/keywords_clean.csv', index_col=0)
df['Earliest_update'] = pd.to_datetime(df.Earliest_update)
df['Latest_update'] = pd.to_datetime(df.Latest_update).dt.tz_localize(None)
df

In [ ]:
df[['Keywords', 'Latest_update']].values.tolist()

In [ ]:
old_earliest_start = pd.to_datetime(df['Earliest_update'].unique()[0])

In [ ]:
old_earliest_start

In [ ]:
new_earliest_start = datetime.strptime('2022-03-01', '%Y-%m-%d')

In [ ]:
new_end = datetime.strptime('2022-04-01', '%Y-%m-%d')

In [ ]:
ls = scraper.splitter(kw_list=df[['Keywords', 'Latest_update']].values.tolist(),
                      char_limit=scraper.QUERY_CHAR_LIMIT,
                      earliest_start=new_earliest_start)

In [ ]:
for l in ls:
    print(l[1])

In [ ]:
dump_path = 'data/dump/2022-03-01__2022-04-01/'

In [ ]:
with open(dump_path + 'dump.json', 'rb') as fin:
    all_tweets = json.load(fin)

In [ ]:
len(all_tweets)

In [ ]:
#all_tweets = []
client = scraper.create_client(scraper.init())

for sublist, start_time in ls:
    res, max_t = scraper.get_tweets(client=client,
                                    keywords=sublist,
                                    start_time=start_time,
                                    end_time=new_end)

    print(len(res), 'tweets')

    all_tweets += res

    with open(dump_path + 'dump.json', 'w') as fout:
        json.dump(all_tweets, fout)

    df.loc[df.Keywords.isin(sublist), 'Latest_update'] = np.fmax(pd.Series([max_t]*len(sublist)).values, df.loc[df.Keywords.isin(sublist), 'Latest_update'].values)
    df.loc[df.Keywords.isin(sublist), 'Earliest_update'] = np.fmin(pd.Series([new_earliest_start]*len(sublist)).values, df.loc[df.Keywords.isin(sublist), 'Earliest_update'].values)
    
df.to_csv('data/keywords_clean.csv')

In [ ]:
id_tweet_dict = {}

for tweet in all_tweets:
    if tweet['id'] in id_tweet_dict.keys():
        id_tweet_dict[tweet['id']].append(tweet)
    else:
        id_tweet_dict[tweet['id']] = [tweet]

In [ ]:
len(id_tweet_dict)

In [ ]:
res = [v[0] for v in id_tweet_dict.values()]

In [ ]:
with open(dump_path + 'clean.json', 'w') as fout:
    json.dump(res, fout)